In [1]:
import clusters
import utils.plotting2D
import utils.transforms2D
import copy

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

import gicp
import gicp_linear

from sklearn.neighbors import NearestNeighbors

# import gtsam
# import os
# from gtsam.symbol_shorthand import L, X

#import gicp

# Define point clouds

In [3]:

np.random.seed(seed=2)

classLabels = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigmarange = (-0.5,0.5)

cluster1 = clusters.cluster2()
cluster1.fillClusterRandomly(20,xrange, yrange, sigmarange, classLabels)

rot = utils.transforms2D.R2(np.radians(90))
t = np.array([[5],
              [0]])
cluster2 = copy.deepcopy(cluster1)
cluster2.transform(rot,t, transformCov = True)

ax, _ = utils.plotting2D.prepAxes()
cluster1.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'b', markerShape = 'o')
cluster2.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'r', markerShape = 'o')

# Rotating covariances

In [6]:
test = np.array([np.eye(2), np.array([[1,2],[3,4]])])
r = utils.transforms2D.R2(np.radians(90))

test2 = np.zeros_like(test)
for i,m in enumerate(test):
    test2[i] = r @ m @ r.T

tmp = (r @ test[:,None,:] @ r.T).squeeze()
print(tmp-test2)

[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]


# Check Nearest Neighbors

In [11]:
np.random.seed(seed=1)

classLabels = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigmarange = (-0.5,0.5)

cluster1 = clusters.cluster2()
cluster1.fillClusterRandomly(4,xrange, yrange, sigmarange, classLabels)

rot = utils.transforms2D.R2(np.radians(0))
t = np.array([[0.2],
              [0]])
cluster2 = copy.deepcopy(cluster1)
cluster2.transform(rot,t, transformCov = True)

ax, _ = utils.plotting2D.prepAxes()
cluster1.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'b', markerShape = 'o')
cluster2.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'r', markerShape = 'o')
 
neigh = NearestNeighbors(n_neighbors = 1)
neigh.fit(cluster1.points.reshape(-1,2))
i = neigh.kneighbors(cluster2.points.reshape(-1,2), return_distance = False)
print(i)

[[0]
 [1]
 [2]
 [3]]


# General ICP

In [3]:
np.random.seed(seed=2)

classLabels = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigmarange = (-0.5,0.5)

cluster1 = clusters.cluster2()
cluster1.fillClusterRandomly(3,xrange, yrange, sigmarange, classLabels)

x_true = np.array([5,0,0])
R_true, t_true = utils.transforms2D.x_to_Rt(x_true)
cluster2 = copy.deepcopy(cluster1)
cluster2.transform(R_true,t_true, transformCov = True)

invx_est, fmin, itr =  gicp.generalICP(cluster2.points, cluster2.covariances, #source
                                        cluster1.points,cluster1.covariances, #target
                                        n_iter_max = 200,
                                        x0 = np.zeros(3))
                                        #x0 = utils.transforms2D.inverse_x(x_true))
print(invx_est)
print(fmin)
print(itr)

invEst_R, invEst_t = utils.transforms2D.x_to_Rt(invx_est)

cluster2.transform(invEst_R,invEst_t, transformCov = True)

ax, _ = utils.plotting2D.prepAxes()
cluster1.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'b', markerShape = 'o')
cluster2.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'r', markerShape = 'o')

[-3.2719441   2.58450062 -0.71800413]
5.812957005288691
200


# linear General ICP

In [3]:
np.random.seed(seed=2)

classLabels = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigmarange = (-0.5,0.5)

cluster1 = clusters.cluster2()
cluster1.fillClusterRandomly(10,xrange, yrange, sigmarange, classLabels)

x_true = np.array([5,0,np.pi/2])
R_true, t_true = utils.transforms2D.x_to_Rt(x_true)
cluster2 = copy.deepcopy(cluster1)
cluster2.transform(R_true,t_true, transformCov = True)

invx_est, fmin, itr, df = gicp_linear.gicp_linear(cluster2.points, cluster2.covariances, #source
                                        cluster1.points,cluster1.covariances, #target
                                        n_iter_max = 500,
                                        x0 = np.zeros(3))
                                        #x0 = utils.transforms2D.inverse_x(x_true))

x_est = utils.transforms2D.inverse_x(invx_est)
print(x_est)
print(fmin)
print(itr)
print(df)

invEst_R, invEst_t = utils.transforms2D.x_to_Rt(invx_est)

cluster2.transform(invEst_R,invEst_t, transformCov = True)

ax, _ = utils.plotting2D.prepAxes()
cluster1.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'b', markerShape = 'o')
cluster2.plot(ax = ax, plotIndex = True,plotCov = True, markerSize = 30, markerColor = 'r', markerShape = 'o')

SyntaxError: positional argument follows keyword argument (2296495738.py, line 9)